In [ ]:
import numpy as np
from scipy.linalg import expm
from qiskit import QuantumCircuit, transpile, assemble
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram

# Parámetros del problema
L = 1  # Longitud del pozo de potencial
V0 = 1  # Potencial fuera del pozo de potencial
N = 4  # Número de puntos en los que dividimos el espacio

# Parámetros de la simulación
timesteps = 100  # Número de pasos de tiempo
dt = 0.1  # Incremento de tiempo

# Definición del Hamiltoniano para el sístema
# en este caso, el Hamiltoniano va a ser uno formado
#por dos hamiltonianos locales
#Es decir H=\sum_{i=1}^{i=L}H_{l}, donde justamente
# H_{l} son los hamiltonianos locales
def local_hamiltonians():
    # Definición de la matriz Hamiltoniana aquí
    #Aquí se definen los hamiltonianos locales
    #para este caso, tomaremos solo dos hamiltonianos locales
    H1 = np.array([[1, -1], [-1, 1]]) #Hamiltoniano local 1
    H2 = np.array([[2, -2], [-2, 2]]) #Hamiltoniano local 2
    return [H1, H2]

# Función para aplicar el operador de evolución temporal trotterizado
def trotterized_time_evolution_operator(circuit, dt):
    hbar=1 #Asumiendo hbar=1 por simplicidad
    local_Hs=local_hamiltonians() # se asigna el valor de la función local_hamiltonian a la variable local_Hs
    L=len(local_Hs) # Este es el número de hamiltonianos locales;2 son los que aparecen como cota superior de la suma

    m=100 #número de pasos para trotterizar
    
    for _ in range(m):  # Aplicar los pasos para Trotterizar
        for H in local_Hs:
            U = expm(-1j * H * dt / (hbar * m))  # Paso simple de trotter para el hamiltoniano local actual
            circuit.unitary(U, range(num_qubits))  # Aplicar la matriz unitaria al circuito cuántico

# Crear circuito cuántico
num_qubits = int(np.log2(N))
qc = QuantumCircuit(num_qubits, num_qubits)

# Inicializar el estado del electrón en una superposición
qc.h(range(num_qubits))

# Evolución temporal
for _ in range(timesteps):
   trotterized_time_evolution_operator(qc, dt)

# Medir los qubits
qc.measure(range(num_qubits), range(num_qubits))

n=1000
# Simular el circuito cuántico
simulator = Aer.get_backend('qasm_simulator')
result = transpile(qc, simulator)

# unir el circuito cuántico en un Qobj 
qobj=assemble(result)

# Correr el circuito cuántico simulado sobre el backend
results= simulator.run(result).result()

# recibir los conteos de las medidas
counts =results.get_counts(result)

# Calcular las probabilidades de encontrar el electrón en diferentes regiones del espacio
probabilities = {}
for key, value in counts.items():
    position = int(key, 2) * L / (2 ** num_qubits)
    if position in probabilities:
        probabilities[position] += value / n
    else:
        probabilities[position] = value / n
        
# Mostrar resultados
print("Probabilidades de encontrar el electrón en diferentes posiciones:")
print(probabilities)
plot_histogram(counts)